# **Real-Time Plant Health Check!**

Having built our classification model by using MobileNetV2 as our base of CNN, we will not try to load and implement the same model to perform classification real-time using Webcamera and Android mobile camera

# Import all the modules

In [ ]:
import keras
keras.__version__

'2.7.0'

In [ ]:
import cv2
from PIL import Image
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import urllib.request

import tensorflow as tf
from keras.preprocessing import image
from keras import models
from keras import layers
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

***********************************************************************************************


# **Implement Model to perform real-time classification**

We will impement model on webcam and android camera. Unfortunately I have not been able to access IOS cameras because of the security encryption and permissions. Furthermore, the interface still requires laptops to function and run prediction. Using Kivy to develop a UI with python to make an applcation could be one way to improve this and will be undertaken as a separate project.

That said, lets with the integration!

Lets first load our trained model.

In [ ]:
# Load the model that we just trained
model = keras.models.load_model('C:/Users/rishi/Documents/MSBAPM/SEM 4/Deep Learning/CNN Project/model_3.h5')

We will first need to create a directory to store the images captured by our cameras

**NOTE**

Please make the following folders before you run the following code

* Main path "Path" and store the h5 file in it
* Folder "Path/Data/"
* Sub folder "Path/Data/input_image_android/" and "Path/Data/input_image_web/" for the screen grabs 

In [ ]:
capture_data = 'C:/Users/rishi/Documents/MSBAPM/SEM 4/Deep Learning/CNN Project/Data/'

>These images need to be processed to make them compatible to act as our input for prediction. I am creating a helper function that I will leverage while making predictions.

In [ ]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 160x160x3
    img = tf.image.resize(img, (160,160))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    
    # Return image
    return img

_________________________

# **Next the main crux of this project!! Integrating our laptop or mobile camera**

 For the first part, I will integrate this with just the webcam as this solution will be usefull for those without an Android phone. For my use case this is not the best option but for other practical business problems such as Facial Verification and Face tagging, this would be all you need! 
 
 But before we dive in, lets get familiar with openCV to understand what exactly is happening here.

In [ ]:
cap = cv2.VideoCapture(0)   # Start an Open CV instance that accesses your webcam. Change from 0 if there are multiple cameras
while cap.isOpened():  # While camera is open
    ret, frame = cap.read() # get the frame that is being captured
    frame = frame[120:120+250,200:200+250, :]  # Resize the frame to record only a section of the screen(here it is 250x250)
    
    cv2.imshow('Verification', frame)  # Show our frame on screen
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  # wait for 1 millisecond after hitting 'q' to exit
        break
cap.release()   # release the webcam
cv2.destroyAllWindows()  # stop presenting the frame 

## **With a similar trigget key, we can do the following**

1. Capture a single frame
1. Store it to our directory
1. Read it and process it in model compatible format
1. Evaluate and predict class
1. Print class

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(1) & 0xFF == ord('v'):
        # Save input image to capture_data/input_image folder 
        cv2.imwrite(os.path.join(capture_data, 'input_image_web', 'input_image.jpg'), frame) # Write the screen grab to our directory
        
        # Run model
        grab = preprocess(os.path.join(capture_data, 'input_image_web', 'input_image.jpg')) # Make image compatible with model
        grab = tf.reshape(grab, (1,160,160,3))
        pred = np.round(model.predict(grab),0) # predict the probability and then predict class from probability
        if pred==0:
            print('Healthy')
        else:
            print('Unhealthy')
        
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Unhealthy
Unhealthy
Unhealthy
Unhealthy
Unhealthy


----------------------------------------------------------

# **Accesing Mobile Camera to perform classification**

This is great but not commpletely practical. No one is carrying around their laptops to identify. A work around to this is using an application that streams video capture screen of your mobile to a website and then get access to this stream to get a snapshot of our camera screen. 
I will try to access our mobiles camera to perform our prediction. We will need to do the following as a part of our setup.

**The following steps and chunk of code will require access to your IP address, Please use it at your own discretion**

1. On your Android Device, download the IP Webcam application
> Link:- https://play.google.com/store/apps/details?id=com.pas.webcam&hl=en_US&gl=US

1. Scroll down and start server. Copy the IP address found at the bottom of your screen

1. Paste it in the folowing block of code


For experimentation, try the next code out and have fun!

In [ ]:
URL = "http://PASTE YOUR IP ADDRESS HERE/shot.jpg"

while True:
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    img = cv2.imdecode(img_arr,-1)
    img = img[340:340+400,760:760+400, :]
    img = np.rot90(img, k=3, axes=(0, 1))
    cv2.imshow('IPWebcam',img)
    
    q = cv2.waitKey(1)
    if q == ord("q"):
        break;

    
cv2.destroyAllWindows()

## **Predict Cases**
Now lets get to business, the following code will use the same concept but this time we will introduce another key to grab a stil image from our stream. Not the fanciest ways to capture a picture but it suffices for the scope of this project. After this image is captured, we would like to read it and pass it through our model to give us a prediction. This is done as follows

In [ ]:
URL = "http://PASTE YOUR IP ADDRESS HERE/shot.jpg"

while True:
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    img = cv2.imdecode(img_arr,-1)
    img = img[340:340+400,760:760+400, :]
    img = np.rot90(img, k=3, axes=(0, 1))
    cv2.imshow('IPWebcam',img)
    
    v = cv2.waitKey(1)
    if v == ord("v"):     # Use key "v" to capture an image
        cv2.imwrite(os.path.join(application_data, 'input_image_android', 'input_image.jpg'), img) # Write the screen grab to our directory
        
        # Run model
        grab_2 = preprocess(os.path.join(application_data, 'input_image_android', 'input_image.jpg')) # Make image compatible with model
        grab_2 = tf.reshape(grab_2, (1,160,160,3)) # reshape it as a single image input 
        pred = np.round(model.predict(grab_2),0) # predict the probability and then predict class from probability
        if pred==0: # Print predicted class
            print('Healthy') 
        else:
            print('Unhealthy')
            
    
    q = cv2.waitKey(1)
    if q == ord("q"):
        break;

    
cv2.destroyAllWindows()

# **Improvements**

Model was trained initially using MobileNetV2 under the asssumption that we would need to deploy it on mobile processors. But with the current application, we can experiment with more involved pretrained models.